## Model validation

The [previous notebook](03-a-model-validation-workflow.ipynb) demonstrated the workflow to validate the random forest pixel classifier. Here, all images of the test dataset will be used to validate the classifier.

All three annotated Z-layers of an ground truth image will be compared to its corresponding prediction. Afterwards, the sum of true positive, true negative, false positive, and false negative values will be calculated. Out of these sums the accuracy, precision, recall, F1-score, and Jaccard Index will be calculated.

The ground truth annotation were annotated from 2 different people with different annotation styles. Here, annotation of person A will be compared to the prediction.

In [5]:
import apoc
import numpy as np
import os
import pandas as pd
import pyclesperanto_prototype as cle

from the_segmentation_game import metrics

In [2]:
# Load the classifier
quapos_lm = apoc.ObjectSegmenter(opencl_filename="02-quapos-lm.cl")

# Show if correctly loaded
quapos_lm.feature_importances()

{'gaussian_blur=1': 0.32557488170342097,
 'difference_of_gaussian=1': 0.4231073391932076,
 'laplace_box_of_gaussian_blur=1': 0.25131777910337144}

In [12]:
# Define File list with images and corresponding ground truth
test_data = "../data/02-data-for-pixel-classifier/test-data/d-images-for-validation/"
annotation_data = "../data/02-data-for-pixel-classifier/test-data/e-annotation-ground-truth-person-a/"

In [13]:
# Define a file list
file_list = os.listdir(test_data)
print(file_list)

['e2.tif', 'f2.tif', 'g2.tif', 'i2.tif', 'k2.tif', 'l2.tif', 's2.tif']


In [15]:
# Create empty array for storing confusion matrix results
confusion_matrix = []

# Itterate a for loop over all images
for i, file_name in enumerate(file_list):
    
    # Open the normalised image
    normalised_image = cle.imread(test_data + file_name)
    prediction = quapos_lm.predict(normalised_image)
    ground_truth = cle.imread(annotation_data + file_name)
    
    # Itterate for loop over Z-layers
    for z_layer in range(0, normalised_image.shape[0]):
        
        # Only compute confusion matrix from annotated slice
        if np.max(ground_truth[z_layer, ...]) > 0:
            
            # Compute confusion matrix
            values = metrics.compute_tp_tn_fp_fn(reference_label_image=ground_truth[z_layer, ...],
                                                 test_label_image=prediction[z_layer, ...])
            
            # Store values to defined empty array
            confusion_matrix.append(values)
            
# Turn confusion matrix into pandas dataframe
confusion_matrix = pd.DataFrame(confusion_matrix)

# Rename the column names
confusion_matrix.rename(columns={0: "tp", 1: "tn", 2: "fp", 3: "fn"}, inplace=True)

3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3 2 lens
3

In [16]:
confusion_matrix

,tp,tn,fp,fn
0,180.0,8636.0,8.0,76.0
1,343.0,8460.0,22.0,75.0
2,65.0,8761.0,66.0,8.0
3,158.0,27612.0,26.0,28.0
4,534.0,27161.0,77.0,52.0
5,647.0,26959.0,136.0,82.0
6,397.0,15443.0,111.0,39.0
7,342.0,15443.0,171.0,34.0
8,256.0,15624.0,70.0,40.0
9,209.0,17012.0,56.0,13.0


In [17]:
#calculating the sum of all measurements
confusion_matrix_sum = np.sum(confusion_matrix)

#calculate accuracy
accuracy = (confusion_matrix_sum["tp"] +
            confusion_matrix_sum["tn"]) / (confusion_matrix_sum["tp"] +
                                           confusion_matrix_sum["tn"] +
                                           confusion_matrix_sum["fp"] +
                                           confusion_matrix_sum["fn"])

#calculate precision
precision = confusion_matrix_sum["tp"] / (confusion_matrix_sum["tp"] + 
                                          confusion_matrix_sum["fp"])

#calculate recall
recall = confusion_matrix_sum["tp"] / (confusion_matrix_sum["tp"] + 
                                       confusion_matrix_sum["fn"])

#calculate jaccard index
jaccard_index = confusion_matrix_sum["tp"] / (confusion_matrix_sum["tp"] + 
                                              confusion_matrix_sum["fp"] +
                                              confusion_matrix_sum["fn"])
                                          
#calculate f1_score
f1_score = 2 * ((precision * recall) / (precision + recall))

#add the calculation to the array
confusion_matrix_sum["precision"] = precision.tolist()
confusion_matrix_sum["recall"] = recall.tolist()
confusion_matrix_sum["accuracy"] = accuracy.tolist()
confusion_matrix_sum["jaccard_index"] = jaccard_index.tolist()
confusion_matrix_sum["f1_score"] = f1_score.tolist()

#show results
confusion_matrix_sum

tp                 7560.000000
tn               379658.000000
fp                 2004.000000
fn                  994.000000
precision             0.790464
recall                0.883797
accuracy              0.992317
jaccard_index         0.716045
f1_score              0.834529
dtype: float32